In [57]:
import os
import subprocess as sp
import yaml
import shutil

# Convert notebooks to html

In [58]:
os.chdir('/Users/ginoprasad/ginoprasad.github.io')

In [59]:
metadata_path = 'metadata.yaml'

In [60]:
with open(metadata_path) as infile:
    metadata = yaml.safe_load(infile)

In [61]:
for project_notebook_path in metadata['Projects']:
    if not os.path.exists(project_notebook_path):
        print(f"REMOVING {project_notebook_path}")
        metadata['Projects'].remove(project_notebook_path)
        with open(metadata_path, 'w') as outfile:
            yaml.dump(metadata, outfile, default_flow_style=False)

In [62]:
metadata

{'GitHub': 'https://github.com/GinoP123',
 'LinkedIn': 'https://www.linkedin.com/in/ginoprasad',
 'CV': '/Users/ginoprasad/_Receipts/pdf_files/Gino_Prasad_CV.pdf',
 'Projects': ['/Users/ginoprasad/ai_stuff/autotune_phase_vocoder/Autotune_MUS_15.ipynb',
  '/Users/ginoprasad/eng_math/fast_fourier_transform.ipynb',
  '/Users/ginoprasad/ai_stuff/unet_implementation.ipynb',
  '/Users/ginoprasad/ai_stuff/text_classifier/Multilayer Neural Network.ipynb',
  '/Users/ginoprasad/ai_stuff/text_classifier/Image Segmenter.ipynb',
  '/Users/ginoprasad/ai_stuff/text_classifier/Handwriten Text Parser.ipynb',
  '/Users/ginoprasad/ai_stuff/Gaussian Blur Test.ipynb',
  '/Users/ginoprasad/ai_stuff/audio_transcriber/play_note.ipynb',
  '/Users/ginoprasad/eng_math/interpolation.ipynb',
  '/Users/ginoprasad/ai_stuff/edge_detector.ipynb']}

In [63]:
temp_path = f'{os.getcwd()}/projects/temp.html'

In [64]:
max_base_filename_length = 50

In [65]:
project_names, project_paths = [], []
for project_notebook_path in metadata['Projects']:
    print(f"Converting {project_notebook_path}")
    sp.run(f"jupyter nbconvert --to html '{project_notebook_path}' --output '{temp_path}'", shell=True)
    
    
    title_line = sp.run(f"grep '<h1' '{temp_path}'", shell=True, capture_output=True).stdout.decode().split('\n')[0]
    project_name = title_line[title_line.index('>')+1:]
    project_name = project_name[:project_name.index('<')]
    print(f'Project Name: {project_name}')

    project_base_path = os.path.basename(project_notebook_path)[:-len('.ipynb')]
    while len(project_base_path) > max_base_filename_length:
        project_base_path = ' '.join(project_base_path.split(' ')[:-1])
    
    if not project_base_path:
        print(f"\n\n\n\n\tWarning: Project '{project_name}' Name exceeds recommended length\n\n\n\n")
        project_base_path = project_name
    print(f'Project Path: {project_base_path}')
    project_path = f'{os.getcwd()}/projects/{project_base_path}.html'
    
    assert project_path not in project_paths
    os.rename(temp_path, project_path)
    
    project_names.append(project_name)
    project_paths.append(project_path)
    print('\n')

Converting /Users/ginoprasad/ai_stuff/autotune_phase_vocoder/Autotune_MUS_15.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/ai_stuff/autotune_phase_vocoder/Autotune_MUS_15.ipynb to html
[NbConvertApp] Writing 589022 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html


Project Name: MUS 15 Project 1: Creating Autotune From Scratch Using Phase Vocoders
Project Path: Autotune_MUS_15


Converting /Users/ginoprasad/eng_math/fast_fourier_transform.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/eng_math/fast_fourier_transform.ipynb to html
[NbConvertApp] Writing 824735 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html


Project Name: Fast Fourier Transform Implementation
Project Path: fast_fourier_transform


Converting /Users/ginoprasad/ai_stuff/unet_implementation.ipynb


...

[NbConvertApp] Converting notebook /Users/ginoprasad/eng_math/interpolation.ipynb to html
[NbConvertApp] Writing 585324 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html


Project Name: Lagrange Interpolation and Gregory-Newton Interpolation
Project Path: interpolation


Converting /Users/ginoprasad/ai_stuff/edge_detector.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/ai_stuff/edge_detector.ipynb to html


Project Name: Image Edge Detector
Project Path: edge_detector




[NbConvertApp] Writing 590662 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html


In [66]:
index_html_path = 'index.html'

In [67]:
index_html_lines = open(index_html_path).readlines()

In [68]:
project_list_index_start = index_html_lines.index('<ul>\n') + 1
project_list_index_end = index_html_lines.index('</ul>\n')

In [69]:
new_project_list =  [f'\t<li><a href="projects/{os.path.basename(html_path)}">{name}</a></li>\n' for name, html_path in zip(project_names, project_paths)]

In [70]:
new_project_list

['\t<li><a href="projects/Autotune_MUS_15.html">MUS 15 Project 1: Creating Autotune From Scratch Using Phase Vocoders</a></li>\n',
 '\t<li><a href="projects/fast_fourier_transform.html">Fast Fourier Transform Implementation</a></li>\n',
 '\t<li><a href="projects/unet_implementation.html">U-Net Convolutional Neural Network Implementation</a></li>\n',
 '\t<li><a href="projects/Multilayer Neural Network.html">Multilayer Neural Network Implementation From Scratch: Handwritten Text Parser</a></li>\n',
 '\t<li><a href="projects/Image Segmenter.html">Image Segmenter Using Connected Components</a></li>\n',
 '\t<li><a href="projects/Handwriten Text Parser.html">Perceptron Implementation From Scratch: Handwritten Text Parser</a></li>\n',
 '\t<li><a href="projects/Gaussian Blur Test.html">Gaussian Filter Implementation</a></li>\n',
 '\t<li><a href="projects/play_note.html">Pitch Reconstruction with Audio Processing</a></li>\n',
 '\t<li><a href="projects/interpolation.html">Lagrange Interpolation 

In [71]:
index_html_lines = index_html_lines[:project_list_index_start] + new_project_list + index_html_lines[project_list_index_end:]

# Copying CV and Updating Links

In [72]:
assert shutil.copy(metadata['CV'], f"projects/{os.path.basename(metadata['CV'])}")

In [73]:
index_html_lines

['<!DOCTYPE html>\n',
 '<html>\n',
 '<head>\n',
 '<title>Gino Prasad</title>\n',
 '</head>\n',
 '\n',
 '<body>\n',
 '<h1> Gino Prasad </h1>\n',
 '<h2>\n',
 '\tHello, this is my personal website\n',
 '</h2>\n',
 '\n',
 '<br>\n',
 "<a id='GitHub' href='https://github.com/GinoP123'> GitHub </a>\n",
 '<br>\n',
 "<a id='CV' href='projects/Gino_Prasad_Resume.pdf'> CV </a>\n",
 '<br>\n',
 "<a id='LinkedIn' href='https://www.linkedin.com/in/ginoprasad'> LinkedIn </a>\n",
 '<br><br>\n',
 '\n',
 '<h2> Cool Projects </h2>\n',
 '<ul>\n',
 '\t<li><a href="projects/Autotune_MUS_15.html">MUS 15 Project 1: Creating Autotune From Scratch Using Phase Vocoders</a></li>\n',
 '\t<li><a href="projects/fast_fourier_transform.html">Fast Fourier Transform Implementation</a></li>\n',
 '\t<li><a href="projects/unet_implementation.html">U-Net Convolutional Neural Network Implementation</a></li>\n',
 '\t<li><a href="projects/Multilayer Neural Network.html">Multilayer Neural Network Implementation From Scratch: Han

In [74]:
tag_dict = {tag: metadata[tag] for tag in ['CV', 'LinkedIn', 'GitHub']}
tag_dict['CV'] = f"projects/{os.path.basename(tag_dict['CV'])}"

In [75]:
for i, line in enumerate(index_html_lines):
    for tag in tag_dict:
        prefix = f"<a id='{tag}' href='"
        if line.startswith(prefix):
            print(line.strip())
            new_line = prefix + tag_dict[tag] + line[len(prefix) + line[len(prefix):].index("'"):]
            print(new_line)
            index_html_lines[i] = new_line
    
    if line.startswith(prefix):
        del tag_dict[tag]

<a id='GitHub' href='https://github.com/GinoP123'> GitHub </a>
<a id='GitHub' href='https://github.com/GinoP123'> GitHub </a>

<a id='CV' href='projects/Gino_Prasad_Resume.pdf'> CV </a>
<a id='CV' href='projects/Gino_Prasad_CV.pdf'> CV </a>

<a id='LinkedIn' href='https://www.linkedin.com/in/ginoprasad'> LinkedIn </a>
<a id='LinkedIn' href='https://www.linkedin.com/in/ginoprasad'> LinkedIn </a>



# Writing Updated Index File

In [76]:
with open(index_html_path, 'w') as outfile:
    outfile.write(''.join(index_html_lines))

In [77]:
sp.run(f"cd '{os.getcwd()}'; git add .; git commit -m 'Automated Website Update'; git push origin main", shell=True)

[main 14822d6] Automated Website Update
 6 files changed, 81 insertions(+), 36 deletions(-)
 create mode 100644 projects/Gino_Prasad_CV.pdf
 delete mode 100644 projects/Gino_Prasad_Resume.pdf


remote: Resolving deltas: 100% (5/5), completed with 5 local objects.        
To https://github.com/ginoprasad/ginoprasad.github.io
   aeb462d..14822d6  main -> main


CompletedProcess(args="cd '/Users/ginoprasad/ginoprasad.github.io'; git add .; git commit -m 'Automated Website Update'; git push origin main", returncode=0)

# Updating Python Script

In [78]:
sp.run(f"jupyter nbconvert --to script 'update_website.ipynb' --output 'update_website'", shell=True)

[NbConvertApp] Converting notebook update_website.ipynb to script
[NbConvertApp] Writing 3802 bytes to update_website.py


CompletedProcess(args="jupyter nbconvert --to script 'update_website.ipynb' --output 'update_website'", returncode=0)